# 一致性测试

For most problems, multiple flux states can achieve the same optimum and thus we try to obtain a consistent network. By this, we mean that there will be mulitple blocked reactions in the network, which gives rise to this inconsistency. To solve this problem, we use algorithms which can detect all the blocked reactions and also give us consistent networks.

对于大多数问题，多个通量状态可以达到相同的最优值，所以我们可以尝试获得一个一致的网络。以此，我们的意思是网络中可能会有多个被阻止的反应，这导致了不一致性。为了解决这个问题，我们使用算法来检查所有被阻止的反应，给我们一致的网络。

Let us take a toy network, like so:

让我们构建一个玩具网络：

\begin{align}
v_1 &: {} \rightarrow 2A \\
v_2 &: A \leftrightarrow B \\
v_3 &: A \rightarrow D \\
v_4 &: A \rightarrow C \\
v_5 &: C \rightarrow D \\
v_6 &: D \rightarrow
\end{align}

Here, $v_{x}$, where $x \in \{1, 2, \ldots, 6\}$ represent the flux carried by the reactions as shown above.

这里 $v_{x}$, $x \in \{1, 2, \ldots, 6\}$ 表示由以上反应带来的通量

In [1]:
import cobra

In [2]:
test_model = cobra.Model("test_model")
v1 = cobra.Reaction("v1")
v2 = cobra.Reaction("v2")
v3 = cobra.Reaction("v3")
v4 = cobra.Reaction("v4")
v5 = cobra.Reaction("v5")
v6 = cobra.Reaction("v6")

test_model.add_reactions([v1, v2, v3, v4, v5, v6])

v1.reaction = "-> 2 A"
v2.reaction = "A <-> B"
v3.reaction = "A -> D"
v4.reaction = "A -> C"
v5.reaction = "C -> D"
v6.reaction = "D ->"

v1.bounds = (0.0, 3.0)
v2.bounds = (-3.0, 3.0)
v3.bounds = (0.0, 3.0)
v4.bounds = (0.0, 3.0)
v5.bounds = (0.0, 3.0)
v6.bounds = (0.0, 3.0)

test_model.objective = v6

unknown metabolite 'A' created
unknown metabolite 'B' created
unknown metabolite 'D' created
unknown metabolite 'C' created


## 使用 FVA

The first approach we can follow is to use FVA (Flux Variability Analysis) which among many other applications, is used to detect blocked reactions. The `cobra.flux_analysis.find_blocked_reactions()` function will return a list of all the blocked reactions obtained using FVA.

我们可以使用的第一步是 FVA，它在其他各种应用中用来检查被阻止的反应。`cobra.flux_analysis.find_blocked_reactions()` 函数会返回一个使用 FVA 获得的，包含所有被阻止的反应的列表。

In [3]:
cobra.flux_analysis.find_blocked_reactions(test_model)

['v2']

As we see above, we are able to obtain the blocked reaction, which in this case is $v_2$.

正如我们以上看到的那样，我们可以获得被阻止的反应，这里是 $v_2$.

## 使用 FASTCC

The second approach to obtaining consistent network in `cobrapy` is to use FASTCC. Using this method, you can expect to efficiently obtain an accurate consistent network. For more details regarding the algorithm, please see [Vlassis N, Pacheco MP, Sauter T (2014)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003424).

第二种在 `cobrapy` 中获得一致网络的方式是使用 FASTCC。使用这种方法，你可以期待高效获取一个准确的一致的网络。关于此算法的更多细节，详见 [Vlassis N, Pacheco MP, Sauter T (2014)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003424).

In [4]:
consistent_model = cobra.flux_analysis.fastcc(test_model)
consistent_model.reactions

[<Reaction v1 at 0x7fc71ddea5c0>,
 <Reaction v3 at 0x7fc71ddea630>,
 <Reaction v4 at 0x7fc71ddea668>,
 <Reaction v5 at 0x7fc71ddea6a0>,
 <Reaction v6 at 0x7fc71ddea6d8>]

Similar to the FVA approach, we are able to identify that $v_2$ is indeed the blocked reaction.

与 FVA 方法相似，我们可以看到 $v_2$ 是被阻止的反应。